You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

&nbsp;

&nbsp;

***
Read in, merge, and clean the four datasets to make a single combined pandas DataFrame.

Create a single pandas DataFrame called employees_final containing:
- Index: employee_id.
- Columns: Ensure the DataFrame contains only the following columns, in the exact order listed: 
    - employee_first_name, employee_last_name, employee_country, employee_city, employee_street, employee_street_number, emergency_contact, emergency_contact_number, relationship, monthly_salary, team, title, office, office_country, office_city, office_street, office_street_number.
- Assign employees to offices based on their country. For any columns that begin with office, replace missing data with "Remote".

In [47]:
import pandas as pd
# df 04
df_ofc_add = pd.read_csv("datasets/office_addresses.csv")
# df 01 
df_emp_add = pd.read_excel(io="datasets/employee_information.xlsx", sheet_name=0)
col_emp_add = ["employee_id", "employee_first_name", "employee_last_name", "employee_country", "employee_city", "employee_street", "employee_street_number"]
df_emp_add = df_emp_add[col_emp_add]
# df 02
headers = ["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"]
df_emp_emg = pd.read_excel(io="datasets/employee_information.xlsx", sheet_name="emergency_contacts", header=None, names=headers)
col_emp_emg = ["employee_id", "emergency_contact", "emergency_contact_number", "relationship"]
df_emp_emg = df_emp_emg[col_emp_emg]
# df 03
df_emp = pd.read_json("datasets/employee_roles.json")
df_emp = df_emp.swapaxes("index", "columns").reset_index()
df_emp = df_emp.rename(columns={"index":"employee_id"})
col_emp = ["employee_id", "monthly_salary", "team", "title"]
df_emp = df_emp[col_emp]
# df merge
df_0102 = df_emp_add.merge(df_emp_emg, on="employee_id", how="outer")
df_010203 = df_0102.merge(df_emp, on="employee_id", how="outer")
df_01020304 = df_010203.merge(df_ofc_add, left_on="employee_country", right_on="office_country", how="outer")
df_01020304.loc[df_01020304["office"].isna(), df_01020304.columns.str.contains("office")] = "Remote"
employees_final = df_01020304
employees_final = employees_final.set_index("employee_id")
employees_final = employees_final.rename_axis(None)
print(employees_final)
print(employees_final.keys())
# print(df_ofc_add)
# print(df_ofc_add.keys())
# print(df_emp_add)
# print(df_emp_add.keys())
# print(df_emp_emg)
# print(df_emp_emg.keys())
# print(df_emp)
# print(df_emp.keys())

       employee_first_name  ... office_street_number
A2R5H9                 Jax  ...                 38.0
H8K0L6                Tara  ...                207.0
G4R7V0               Gemma  ...                350.0
M1Z7U9                 Tig  ...               Remote

[4 rows x 17 columns]
Index(['employee_first_name', 'employee_last_name', 'employee_country',
       'employee_city', 'employee_street', 'employee_street_number',
       'emergency_contact', 'emergency_contact_number', 'relationship',
       'monthly_salary', 'team', 'title', 'office', 'office_country',
       'office_city', 'office_street', 'office_street_number'],
      dtype='object')
